# Example: Modular Analysis Framework

This notebook demonstrates the new modular scripts for FTIR/HIPS/Aethalometer analysis.

## Key Features:
- **PlotConfig**: Set defaults once, apply to all plots
- **FilterId matching**: Match by physical filter, not just date
- **Flexible layouts**: Individual, grid, or combined plots
- **Site selection**: Plot all sites or specific ones
- **Outlier exclusion**: Traceable, transparent flagging with before/after comparison

## 1. Setup and Imports

In [ ]:
import sys
import os

# Get the directory where the notebook is located and add scripts to path
# This works regardless of what directory you run Jupyter from
notebook_dir = os.path.dirname(os.path.abspath('__file__'))
scripts_path = os.path.join(notebook_dir, 'scripts')
if scripts_path not in sys.path:
    sys.path.insert(0, scripts_path)

# Core imports
from config import SITES, MAC_VALUE
from data_matching import (
    load_aethalometer_data, 
    load_filter_data,
    add_base_filter_id,
    match_by_filter_id,
    match_aeth_filter_data,
    match_hips_with_smooth_raw
)
from flow_periods import (
    add_flow_period,
    has_before_after_data,
    print_flow_period_summary
)

# Outlier system imports
from outliers import (
    EXCLUDED_SAMPLES,
    MANUAL_OUTLIERS,
    apply_exclusion_flags,
    apply_threshold_flags,
    get_clean_data,
    print_exclusion_summary,
    identify_outlier_dates
)

# Plotting imports
from plotting import PlotConfig, crossplots, timeseries, distributions, comparisons

print("Imports successful!")
print(f"Scripts loaded from: {scripts_path}")

## 2. Configure Plot Defaults

Set these once at the top of your notebook. All subsequent plots will use these settings.

In [ ]:
# Option 1: Plot all sites, individual figures
PlotConfig.set(
    sites='all',
    layout='individual',
    figsize=(10, 8),
    show_stats=True,
    show_1to1=True
)

# Show current settings
PlotConfig.show()

In [ ]:
# Option 2: Plot specific sites in a grid
# PlotConfig.set(
#     sites=['Beijing', 'JPL'],
#     layout='grid'
# )

# Option 3: Just one site
# PlotConfig.set(sites='JPL', layout='individual')

## 3. Load Data

In [ ]:
# Load aethalometer data (all sites)
aethalometer_data = load_aethalometer_data()

In [ ]:
# Load filter data
filter_data = load_filter_data()

# Add base_filter_id for proper matching
filter_data = add_base_filter_id(filter_data)
print(f"\nAdded base_filter_id column")
print(f"Example: {filter_data['FilterId'].iloc[0]} -> {filter_data['base_filter_id'].iloc[0]}")

## 4. Match Data by FilterId (Recommended)

This ensures you're comparing measurements from the **same physical filter**.

In [ ]:
# Match FTIR EC and HIPS by FilterId for each site
matched_by_filter = {}

for site_name, config in SITES.items():
    site_code = config['code']
    
    matched = match_by_filter_id(
        filter_data, 
        site_code=site_code,
        params=['EC_ftir', 'HIPS_Fabs', 'ChemSpec_Iron_PM2.5']
    )
    
    if matched is not None:
        # Convert HIPS to BC equivalent (divide by MAC)
        matched['hips_fabs'] = matched['hips_fabs'] / MAC_VALUE
        matched_by_filter[site_name] = matched
        print(f"{site_name}: {len(matched)} filters with matched data")
    else:
        print(f"{site_name}: No matched data")

## 5. Time Series Plots

In [ ]:
# BC time series - uses PlotConfig defaults (all sites, individual)
timeseries.bc(aethalometer_data, wavelength='IR')

In [ ]:
# Override to show all sites in a grid
timeseries.bc(aethalometer_data, wavelength='IR', layout='grid')

In [ ]:
# Multi-wavelength BC for each site
timeseries.bc_multiwavelength(aethalometer_data, sites=['JPL', 'Beijing'])

In [ ]:
# Flow ratio over time
timeseries.flow_ratio(aethalometer_data, layout='grid')

## 6. Cross-Plots (Scatter)

In [ ]:
# HIPS vs FTIR EC - should be close to 1:1 if same physical filter
results = crossplots.scatter(
    matched_by_filter,
    x_col='ftir_ec',
    y_col='hips_fabs',
    xlabel='FTIR EC (µg/m³)',
    ylabel='HIPS Fabs / MAC (µg/m³)',
    title='FTIR EC vs HIPS (same filter)',
    layout='grid'
)

In [ ]:
# With iron as color gradient
crossplots.with_iron_gradient(
    matched_by_filter,
    x_col='ftir_ec',
    y_col='hips_fabs',
    xlabel='FTIR EC (µg/m³)',
    ylabel='HIPS Fabs / MAC (µg/m³)',
    title='FTIR vs HIPS (colored by Iron)',
    sites=['JPL']  # Just one site for this example
)

## 7. Match Aethalometer with HIPS (including smooth/raw info)

In [ ]:
# Match HIPS with aethalometer data (by date)
hips_aeth_matched = {}

for site_name, config in SITES.items():
    if site_name not in aethalometer_data:
        continue
        
    matched = match_hips_with_smooth_raw(
        site_name,
        aethalometer_data[site_name],
        filter_data,
        config['code']
    )
    
    if matched is not None:
        hips_aeth_matched[site_name] = matched
        print(f"{site_name}: {len(matched)} matched pairs")

In [ ]:
# HIPS vs Aethalometer crossplot
crossplots.scatter(
    hips_aeth_matched,
    x_col='ir_bcc',
    y_col='hips_fabs',
    xlabel='Aethalometer IR BCc (µg/m³)',
    ylabel='HIPS Fabs / MAC (µg/m³)',
    title='HIPS vs Aethalometer',
    layout='grid'
)

## 8. Distributions

In [ ]:
# BC distribution boxplot across sites
distributions.bc_boxplot(aethalometer_data, wavelength='IR')

In [ ]:
# Smooth/raw difference histogram
distributions.smooth_raw_histogram(
    hips_aeth_matched,
    col='smooth_raw_abs_pct',
    thresholds=[1, 2.5, 4, 5],
    layout='grid'
)

## 9. Flow Period Analysis

In [ ]:
# Show which sites have before/after data
print_flow_period_summary()

In [ ]:
# Add flow period to matched data
for site_name, df in hips_aeth_matched.items():
    hips_aeth_matched[site_name] = add_flow_period(df, site_name, date_col='date')
    periods = hips_aeth_matched[site_name]['flow_period'].value_counts()
    print(f"{site_name}: {dict(periods)}")

In [ ]:
# Flow period comparison (only for sites with before/after data)
# JPL is the only site with data in both periods
if 'JPL' in hips_aeth_matched:
    comparisons.flow_periods(
        {'JPL': hips_aeth_matched['JPL']},
        x_col='ir_bcc',
        y_col='hips_fabs',
        period_col='flow_period',
        xlabel='Aethalometer IR BCc (µg/m³)',
        ylabel='HIPS Fabs / MAC (µg/m³)'
    )

## 10. Threshold Analysis

In [ ]:
# Analyze effect of smooth/raw thresholds
comparisons.threshold_analysis(
    hips_aeth_matched,
    x_col='ir_bcc',
    y_col='hips_fabs',
    threshold_col='smooth_raw_abs_pct',
    thresholds=[1, 2.5, 4, 5],
    sites=['JPL'],  # Just one site for clarity
    xlabel='Aethalometer IR BCc (µg/m³)',
    ylabel='HIPS Fabs / MAC (µg/m³)'
)

## 11. Outlier Exclusion System

The outlier system provides **traceable, transparent** exclusion of problematic data points:
- **Date-based exclusions**: Specific dates with known issues (contamination, equipment failure)
- **Threshold-based outliers**: Automatic flagging based on value thresholds
- **Never deletes data**: Just flags for masking - easy to compare with/without

In [ ]:
# View current outlier definitions
print("=" * 60)
print("DATE-BASED EXCLUSIONS (in outliers.py)")
print("=" * 60)
for site, exclusions in EXCLUDED_SAMPLES.items():
    print(f"\n{site}: {len(exclusions)} exclusions")
    for exc in exclusions[:3]:  # Show first 3
        print(f"  - {exc['date']}: {exc['reason'][:50]}...")

In [ ]:
# View threshold-based outlier rules
print("=" * 60)
print("THRESHOLD-BASED OUTLIER RULES (in outliers.py)")
print("=" * 60)
for site, config in MANUAL_OUTLIERS.items():
    print(f"\n{site}:")
    print(f"  {config['description']}")
    for criteria in config['remove_criteria']:
        print(f"  Rule: {criteria}")

### 11.1 Apply Outlier Flags to Matched Data

The pipeline adds two boolean columns:
- `is_excluded`: Date-based exclusions
- `is_outlier`: Threshold-based outliers

In [ ]:
# Apply outlier flags to the HIPS-Aethalometer matched data
hips_aeth_with_outliers = {}

for site_name, df in hips_aeth_matched.items():
    df_flagged = df.copy()
    
    # Create columns needed for threshold detection (convert ug/m3 to ng/m3)
    df_flagged['aeth_bc'] = df_flagged['ir_bcc'] * 1000
    df_flagged['filter_ec'] = df_flagged['hips_fabs'] * 1000  # Using HIPS as proxy
    
    # Step 1: Apply date-based exclusions
    df_flagged = apply_exclusion_flags(df_flagged, site_name)
    
    # Step 2: Apply threshold-based outlier flags
    df_flagged = apply_threshold_flags(df_flagged, site_name)
    
    # Step 3: Create combined mask
    df_flagged['is_any_outlier'] = df_flagged['is_excluded'] | df_flagged['is_outlier']
    
    hips_aeth_with_outliers[site_name] = df_flagged
    
    # Print summary
    print_exclusion_summary(df_flagged, site_name)

### 11.2 Before/After Visualization

The key pattern: show outliers with red X markers, compare regression stats before vs after exclusion.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from scipy import stats

def plot_before_after_outliers(data_dict, x_col, y_col, xlabel, ylabel, title_prefix):
    """
    Create side-by-side before/after plots showing outlier impact.
    
    BEFORE: All data with outliers highlighted as red X
    AFTER: Clean data with regression, outliers faded
    """
    site_colors = {'Beijing': '#1f77b4', 'Delhi': '#ff7f0e', 'JPL': '#2ca02c', 'Addis_Ababa': '#d62728'}
    
    for site_name, df in data_dict.items():
        if len(df) == 0:
            continue
            
        fig, axes = plt.subplots(1, 2, figsize=(14, 6))
        color = site_colors.get(site_name, '#333333')
        
        # Get data arrays
        x_data = df[x_col].values
        y_data = df[y_col].values
        outlier_mask = df['is_any_outlier'].values
        
        # Valid data mask
        valid = ~np.isnan(x_data) & ~np.isnan(y_data)
        clean_valid = valid & ~outlier_mask
        outlier_valid = valid & outlier_mask
        
        # ============ BEFORE PLOT (All data, outliers highlighted) ============
        ax = axes[0]
        
        # Plot clean points
        ax.scatter(x_data[clean_valid], y_data[clean_valid], 
                  c=color, alpha=0.7, s=60, label='Data')
        
        # Overlay outliers as red X
        if outlier_valid.any():
            ax.scatter(x_data[outlier_valid], y_data[outlier_valid],
                      c='red', marker='X', s=200, linewidths=2, 
                      label=f'Outliers ({outlier_valid.sum()})')
        
        # Regression on ALL data
        x_all = x_data[valid]
        y_all = y_data[valid]
        if len(x_all) > 2:
            slope, intercept, r_value, _, _ = stats.linregress(x_all, y_all)
            x_line = np.linspace(x_all.min(), x_all.max(), 100)
            ax.plot(x_line, slope * x_line + intercept, 'b-', lw=2, 
                   label=f'Fit: R²={r_value**2:.3f}, slope={slope:.2f}')
        
        # 1:1 line
        lims = [min(ax.get_xlim()[0], ax.get_ylim()[0]), 
                max(ax.get_xlim()[1], ax.get_ylim()[1])]
        ax.plot(lims, lims, 'k--', alpha=0.5, label='1:1')
        
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.set_title(f'{title_prefix} - {site_name}\nBEFORE (n={valid.sum()})')
        ax.legend(loc='upper left', fontsize=9)
        ax.grid(True, alpha=0.3)
        
        # ============ AFTER PLOT (Clean data, outliers faded) ============
        ax = axes[1]
        
        # Plot clean points
        ax.scatter(x_data[clean_valid], y_data[clean_valid], 
                  c=color, alpha=0.7, s=60, label='Retained')
        
        # Show removed outliers (faded)
        if outlier_valid.any():
            ax.scatter(x_data[outlier_valid], y_data[outlier_valid],
                      c='red', marker='X', s=100, alpha=0.3, 
                      label=f'Removed ({outlier_valid.sum()})')
        
        # Regression on CLEAN data only
        x_clean = x_data[clean_valid]
        y_clean = y_data[clean_valid]
        if len(x_clean) > 2:
            slope, intercept, r_value, _, _ = stats.linregress(x_clean, y_clean)
            x_line = np.linspace(x_clean.min(), x_clean.max(), 100)
            ax.plot(x_line, slope * x_line + intercept, 'g-', lw=2, 
                   label=f'Fit: R²={r_value**2:.3f}, slope={slope:.2f}')
        
        # 1:1 line
        ax.plot(lims, lims, 'k--', alpha=0.5, label='1:1')
        
        ax.set_xlabel(xlabel)
        ax.set_ylabel(ylabel)
        ax.set_title(f'{title_prefix} - {site_name}\nAFTER (n={clean_valid.sum()})')
        ax.legend(loc='upper left', fontsize=9)
        ax.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()

# Run the before/after visualization
plot_before_after_outliers(
    hips_aeth_with_outliers,
    x_col='ir_bcc',
    y_col='hips_fabs',
    xlabel='Aethalometer IR BCc (µg/m³)',
    ylabel='HIPS Fabs / MAC (µg/m³)',
    title_prefix='HIPS vs Aethalometer'
)

### 11.3 Get Clean Data for Further Analysis

Use `get_clean_data()` to extract only non-outlier points for downstream analysis.

In [ ]:
# Extract clean data for each site
clean_data = {}

for site_name, df in hips_aeth_with_outliers.items():
    # get_clean_data() removes both is_excluded and is_outlier flagged rows
    clean_df = get_clean_data(df)
    clean_data[site_name] = clean_df
    
    print(f"{site_name}:")
    print(f"  Original: {len(df)} points")
    print(f"  After exclusions: {len(clean_df)} points")
    print(f"  Removed: {len(df) - len(clean_df)} ({100*(len(df)-len(clean_df))/len(df):.1f}%)\n")

### 11.4 Identifying New Outliers

Use `identify_outlier_dates()` to find dates matching your criteria - output is ready to copy into outliers.py.

In [ ]:
# Example: Find outliers where aethalometer BC > 2000 ng/m3
# The output can be copy-pasted directly into outliers.py EXCLUDED_SAMPLES

if 'JPL' in hips_aeth_with_outliers:
    identify_outlier_dates(
        site_name='JPL',
        matched_df=hips_aeth_with_outliers['JPL'],
        criteria={'aeth_bc_min': 2000}  # ng/m3
    )

### 11.5 Impact Assessment: Compare Before vs After Statistics

In [ ]:
# Compare regression statistics before vs after outlier removal
print("=" * 70)
print("IMPACT ASSESSMENT: Regression Statistics Before vs After")
print("=" * 70)

for site_name, df in hips_aeth_with_outliers.items():
    x_col, y_col = 'ir_bcc', 'hips_fabs'
    
    # All data
    x_all = df[x_col].dropna()
    y_all = df.loc[x_all.index, y_col].dropna()
    common_all = x_all.index.intersection(y_all.index)
    
    # Clean data
    clean_df = get_clean_data(df)
    x_clean = clean_df[x_col].dropna()
    y_clean = clean_df.loc[x_clean.index, y_col].dropna()
    common_clean = x_clean.index.intersection(y_clean.index)
    
    if len(common_all) > 2 and len(common_clean) > 2:
        # Before
        slope_all, _, r_all, _, _ = stats.linregress(
            df.loc[common_all, x_col], df.loc[common_all, y_col])
        
        # After
        slope_clean, _, r_clean, _, _ = stats.linregress(
            clean_df.loc[common_clean, x_col], clean_df.loc[common_clean, y_col])
        
        print(f"\n{site_name}:")
        print(f"  R²:    {r_all**2:.3f} -> {r_clean**2:.3f}  (change: {(r_clean**2 - r_all**2):+.3f})")
        print(f"  Slope: {slope_all:.3f} -> {slope_clean:.3f}  (change: {(slope_clean - slope_all):+.3f})")
        print(f"  N:     {len(common_all)} -> {len(common_clean)}  (removed: {len(common_all) - len(common_clean)})")

## 12. Quick Reference: Available Functions

### PlotConfig
```python
PlotConfig.set(sites='all', layout='grid')  # Set defaults
PlotConfig.show()                            # Show current settings
PlotConfig.reset()                           # Reset to defaults
```

### crossplots
```python
crossplots.scatter(data, x_col, y_col)      # Generic scatter
crossplots.bc_vs_ec(data)                   # Preset: Aeth BC vs Filter EC
crossplots.hips_vs_ftir(data)               # Preset: HIPS vs FTIR
crossplots.with_iron_gradient(data, ...)   # Color by iron concentration
```

### timeseries
```python
timeseries.bc(data, wavelength='IR')        # BC time series
timeseries.bc_multiwavelength(data)         # All wavelengths
timeseries.flow_ratio(data)                 # Flow ratio over time
timeseries.data_completeness(data)          # Data availability
```

### distributions
```python
distributions.bc_boxplot(data)              # BC distribution
distributions.smooth_raw_histogram(data)    # Smooth/raw difference
distributions.correlation_matrix(data, cols) # Correlation heatmap
```

### comparisons
```python
comparisons.before_after_outliers(data)     # Outlier removal impact
comparisons.threshold_analysis(data, ...)   # Test different thresholds
comparisons.flow_periods(data)              # Before/after flow fix
```

### data_matching
```python
add_base_filter_id(filter_data)             # Strip -N suffix
match_by_filter_id(data, site_code, params) # Match by physical filter
match_hips_with_smooth_raw(...)             # HIPS + aethalometer
```

### flow_periods
```python
add_flow_period(df, site_name)              # Add flow_period column
has_before_after_data(site_name)            # Check data availability
print_flow_period_summary()                 # Show all sites status
```

### outliers (NEW)
```python
# View definitions
EXCLUDED_SAMPLES                            # Date-based exclusions dict
MANUAL_OUTLIERS                             # Threshold-based rules dict

# Apply flags (adds is_excluded, is_outlier columns)
apply_exclusion_flags(df, site_name)        # Date-based exclusions
apply_threshold_flags(df, site_name)        # Threshold-based outliers

# Extract data
get_clean_data(df)                          # Returns non-outlier rows only

# Utilities
print_exclusion_summary(df, site_name)      # Show exclusion counts
identify_outlier_dates(site, df, criteria)  # Find dates to add to registry
```

### Outlier Workflow Pattern
```python
# 1. Apply flags
df = apply_exclusion_flags(df, site_name)
df = apply_threshold_flags(df, site_name)

# 2. Create combined mask
outlier_mask = df['is_excluded'] | df['is_outlier']

# 3. Use in plotting
ax.scatter(x[~outlier_mask], y[~outlier_mask], label='Clean')
ax.scatter(x[outlier_mask], y[outlier_mask], c='red', marker='X', label='Outliers')

# 4. Get clean data for analysis
clean_df = get_clean_data(df)
```